In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import xlrd
import io
from zipfile import ZipFile
from datetime import date, timedelta, time, datetime

In [3]:
today_yyyymmdd = date.today().strftime("%Y%m%d")
print(today_yyyymmdd)

20200805


import datetime 
base = datetime.date(2020,2,1)
for i in range(0,10): 
    d=base + datetime.timedelta(i)
    d=d.strftime("%Y%m%d")
    print(f"N029kakutokuruiseki{d}.csv")

from datetime import datetime, timedelta

def date_range(start_date: datetime, end_date: datetime):
    diff = (end_date - start_date).days + 1
    return (start_date + timedelta(i) for i in range(diff))

In [4]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

In [82]:
#ディレクトリ
data_dir = Path(r"C:\Users\crie072\Documents\21_Online")


#Input_file もうやらなくてよい（2020/8/5取込完了）
#①累積分
input_file_1 = "online入会者累積リスト_202001以降.xlsx"

#②承認リスト（当月分）
input_file_2 = "N029（2020年7月入会）.xlsx"

#Output_file
#累積リスト
output_file = f"N29オンライン申込累積リスト" 
print("Output①⇒",output_file)

Output①⇒ N29オンライン申込累積リスト


①累積リスト

print(input_file_1)

df_cum = pd.read_excel(data_dir /r"input"/f"{input_file_1}", dtypes={'会員カード発行区分':str})
df_cum.head()

df_cum = df_cum.astype({'会員カード発行区分':str})

print(df_cum.shape)
print(df_cum.index)
print(df_cum.columns)
print(df_cum.dtypes)

オンライン入会客を特定

df_cum =df_cum[(df_cum["会員カード発行区分"] == "1") 
               & (df_cum["続柄"] == 0) 
               & ~(df_cum["カード提携会社2CD"] == 95) 
               & (df_cum["切替CD"] == 0) 
               & (df_cum["受付部支店"] == 299)]

#含まれてないか確認
print(df_cum["会員カード発行区分"].unique())
print(df_cum["続柄"].unique())
print(df_cum["カード提携会社2CD"].unique())
print(df_cum["切替CD"].unique())
print(df_cum["受付部支店"].unique())
df_cum.shape[0]

②当月分の読込

In [91]:
print(input_file_2)

df = pd.read_excel(data_dir /r"input"/f"{input_file_2}")
df.head()

N029（2020年7月入会）.xlsx


,カード加入日,会員カード発行区分,続柄,カード提携会社1CD,カード提携会社2CD,カード種類CD,カードブランドCD,切替CD,勧誘CD,受付部支店,性別CD,再発行CD,旧カード提携会社1CD,旧カード提携会社2CD,旧カード種類CD,旧カードブランドCD,入会申込番号,満年齢,リボ支払方法CD,勧誘団体CD,提携カード特殊エリア,エンボス内容,カード年会費CD,リボ利用対象CD
0,20200701,1,0,D00,8,31,1,0,3,299,2,0,NaN,NaN,NaN,NaN,155051921910960,46,12.0,7000,40.0,7.000000e+03,2,0
1,20200701,1,0,D00,1,31,1,0,3,299,1,0,NaN,NaN,NaN,NaN,155051921918211,50,52.0,6000,40.0,6.000000e+03,2,1
2,20200701,2,1,D00,8,31,1,0,3,1,1,0,NaN,NaN,NaN,NaN,155051921944480,58,52.0,2000,NaN,2.000310e+09,0,1
3,20200701,1,0,D02,1,2,1,0,3,102,1,0,NaN,NaN,NaN,NaN,155603602468245,36,12.0,2000,NaN,2.000010e+09,2,0
4,20200701,5,0,D00,95,0,0,0,3,1,2,0,NaN,NaN,NaN,NaN,155603602537692,47,NaN,0,NaN,NaN,1,0


オンライン入会客を特定

In [92]:
df =df[(df["会員カード発行区分"] == "1") 
               & (df["続柄"] == 0) 
               & ~(df["カード提携会社2CD"] == 95) 
               & (df["切替CD"] == 0) 
               & (df["受付部支店"] == 299)]

In [93]:
#含まれてないか確認
print(df["会員カード発行区分"].unique())
print(df["続柄"].unique())
print(df["カード提携会社2CD"].unique())
print(df["切替CD"].unique())
print(df["受付部支店"].unique())
df.shape[0]

['1']
[0]
[ 8  1  4 18 14 11]
[0]
[299]


889

In [94]:
print(df.shape)
print(df.index)
print(df.columns)
print(df.dtypes)

(889, 24)
Int64Index([   0,    1,    5,   11,   31,   32,   35,   39,   42,   69,
            ...
            6763, 6767, 6772, 6781, 6782, 6787, 6794, 6801, 6832, 6891],
           dtype='int64', length=889)
Index(['カード加入日', '会員カード発行区分', '続柄', 'カード提携会社1CD', 'カード提携会社2CD', 'カード種類CD',
       'カードブランドCD', '切替CD', '勧誘CD', '受付部支店', '性別CD', '再発行CD', '旧カード提携会社1CD',
       '旧カード提携会社2CD', '旧カード種類CD', '旧カードブランドCD', '入会申込番号', '満年齢', 'リボ支払方法CD',
       '勧誘団体CD', '提携カード特殊エリア', 'エンボス内容', 'カード年会費CD', 'リボ利用対象CD'],
      dtype='object')
カード加入日           int64
会員カード発行区分       object
続柄               int64
カード提携会社1CD      object
カード提携会社2CD       int64
カード種類CD          int64
カードブランドCD        int64
切替CD             int64
勧誘CD             int64
受付部支店            int64
性別CD             int64
再発行CD            int64
旧カード提携会社1CD     object
旧カード提携会社2CD    float64
旧カード種類CD       float64
旧カードブランドCD     float64
入会申込番号           int64
満年齢              int64
リボ支払方法CD       float64
勧誘団体CD           int64
提携カード特殊エリア    

前月と当月の承認リストをドッキング

df_cum = pd.concat([df_cum, df_pre])
df_cum.shape[0]

print(df_cum['カード加入日'].unique())

重複削除

889

CSVアウトプット（上書き）

In [90]:
df_pre.to_csv(data_dir /r"output"/f"{output_file}.csv", mode='a',index=False,header=None,encoding="shift_jis")